## Training Classifiers

Neural networks are very flexible algorithms which have revolutionized artificial intelligence in the last few years. Their success is due to new tricks in designing and training the networks, but also the availability of large datasets and the computing power to process them.

In this notebook we will look at some simple networks and how they deal with the data we prepared earlier. When working with machine learning, it is a good idea to start with simpler models, and only proceed to more sophisticated methods when you know the limitations of the simple models have been reached.

I haven't spent much time tweaking the hyperparameters of these models, so there may be a few additional percent of accuracy here and there, which can be found by tweaking the models a little. More so if a more systematic approach to optimize them was to be used.

Another problem is that the test data was used to optimize the models and their hyperparameters. This introduces a certain amount of "data snooping" and slightly defies the purpose of the out-of-sample performance validation. I decided to make this tradeoff because there is already very few data, and splitting off another test set would have hampered performance further.

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pysam
import random
import feather
import h5py
%matplotlib inline

In [3]:
training_data = feather.read_dataframe("amplicon_training_metadata.feather")
test_data = feather.read_dataframe("amplicon_test_metadata.feather")
h5f = h5py.File("amplicon_dataset.hdf", "r")

In [84]:
training_X = h5f["training/X"][:]
training_y = h5f["training/y"][:]

test_X = h5f["test/X"][:]
test_y = h5f["test/y"][:]

In [77]:
def evaluate_model(model):
    y_hat = model.predict(training_X).argmax(axis=1)
    training_accuracy = (y_hat == training_y.argmax(axis=1)).sum() / len(training_y)

    y_hat = model.predict(test_X).argmax(axis=1)
    test_accuracy = (y_hat == test_y.argmax(axis=1)).sum() / len(test_y)
    
    return  training_accuracy, test_accuracy

In [78]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.core import Dense, Activation, Dropout,  Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.regularizers import l2, activity_l2

## One Relu Layer

The first model consists only of one layer.

In [87]:
model = Sequential()
model.add(Dense(input_dim=200, output_dim=100, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dense(output_dim=11, init="glorot_uniform"))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05, momentum=0.05, nesterov=True))

In [88]:
for i in range(10):
    model.fit(training_X, training_y, nb_epoch=10, batch_size=64,verbose=False)
    print ( "%.2f %.2f" % evaluate_model(model))

0.77 0.55
0.86 0.61
0.80 0.57
0.87 0.58
0.89 0.54
0.84 0.52
0.76 0.48
0.97 0.55
0.98 0.55
0.98 0.54


The first column shows the accuracy on the training set, the second on the test set. As we can see, this model performs stunningly well on the training set, but quite bad on the test set.

This is due to overfitting.

But: Because there are 11 amplicons, 56% isn't disheartiningly bad either..

## Two Layers, with dropout

The second model contains two layers. Dropout is activated for both of them, in order to reduce overfitting.

In [70]:
model = Sequential()
model.add(Dense(input_dim=200, output_dim=100, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(input_dim=200, output_dim=100, init="glorot_uniform"))
model.add(Dropout(0.5))
model.add(Dense(output_dim=11, init="glorot_uniform"))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05, momentum=0.05, nesterov=True))

In [71]:
for i in range(20):
    model.fit(training_X, training_y, nb_epoch=40, batch_size=64,verbose=False)
    print ("%.2f %.2f" % evaluate_model(model))

0.75 0.56
0.80 0.58
0.81 0.58
0.82 0.59
0.83 0.59
0.83 0.58
0.84 0.59
0.83 0.58
0.84 0.58
0.85 0.60
0.85 0.58
0.85 0.58
0.85 0.58
0.85 0.59
0.86 0.59
0.85 0.58
0.86 0.59
0.85 0.57
0.86 0.59
0.85 0.57


This model has a significantly harder time to learn/overfit the training set. In turn the test performance is a bit better, but still not useful.

## Data augmentation

When the training performance is high and the test performance is low, it's often a problem of too little data. But what do we do if we don't have enough data? Well, we can just make it up.

The idea is to just add a bit of noise to the event data.

In [160]:
model = Sequential()
model.add(Dense(input_dim=200, output_dim=200, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(input_dim=200, output_dim=200, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(output_dim=11, init="glorot_uniform"))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05, momentum=0.05, nesterov=True))

In [163]:
def batch_feeder():
    nb = 200
    while 1:        
        index = np.arange(len(training_X))
        np.random.shuffle(index)
        for i in np.arange(0,len(index),nb):
            sub = index[i:i+nb]
            X = training_X[sub,:]
            n,m = X.shape
            X += np.random.normal(0,0.5,size=(n,m)) * np.random.normal(1,0.1) + np.random.normal(0,0.1)
            y = training_y[sub,:]
            yield X, y

In [164]:
for i in range(20):
    model.fit_generator(batch_feeder(), samples_per_epoch=1000000, nb_epoch=1, verbose=0)
    print ("%.2f %.2f" % evaluate_model(model))

0.95 0.78
0.95 0.78
0.96 0.78
0.96 0.78
0.96 0.78
0.97 0.79
0.97 0.78
0.97 0.78
0.97 0.79
0.97 0.79
0.97 0.79
0.97 0.79
0.97 0.79
0.97 0.80
0.97 0.79
0.97 0.80
0.98 0.80
0.98 0.79
0.98 0.79
0.98 0.80


Test performance approaches 80%. Still not good enough, but we are getting there.

## Convolutional Neural Networks

One of the most important "tricks" that led to the boom of "deep learning" is the idea of Convolutional Neural Networks. These networks train small feature detectors, for example with a width of three elements and apply these on the full length of the input multiple times. This achieves translational invariance, which means it can recognize patterns independently of where they are occurring, whereas a normal network can only recognize patterns at exactly the same place where they were learned.

In [274]:
# implementational detail: 1D Convolutional layers in keras expect inputs to have three dimensions

def reshape3(X):
    n,m = X.shape
    Xn = X.copy()
    Xn.shape = (n,m,1)
    return Xn

def batch_feeder():
    nb = 50
    while 1:        
        index = np.arange(len(training_X))
        np.random.shuffle(index)
        for i in np.arange(0,len(index),nb):
            sub = index[i:i+nb]
            X = training_X[sub,:]
            n,m = X.shape
            X += np.random.normal(0,0.5,size=(n,m)) * np.random.normal(1,0.05) + np.random.normal(0,0.1)
            X = reshape3(X)
            y = training_y[sub,:]
            yield (X, y)

training_X3 = reshape3(training_X)[:3000,:]
test_X3 = reshape3(test_X)
def evaluate_model(model):
    y_hat = model.predict(training_X3).argmax(axis=1)
    training_accuracy = (y_hat == training_y[:3000,:].argmax(axis=1)).sum() / 3000

    y_hat = model.predict(test_X3).argmax(axis=1)
    test_accuracy = (y_hat == test_y.argmax(axis=1)).sum() / len(test_y)
    
    return  training_accuracy, test_accuracy

In [288]:
model = Sequential()
model.add(Convolution1D(nb_filter=20,
                        filter_length=3,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1,
                        input_shape=(200,1),
                        W_regularizer= l2(0.01),
                       ))
model.add(Convolution1D(nb_filter=20,
                        filter_length=3,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1,
                        input_shape=(200,1),
                        W_regularizer= l2(0.01),
                       ))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(output_dim=50, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dense(output_dim=11, init="glorot_uniform"))
model.add(Activation("softmax"))

In [289]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05, momentum=0.05, nesterov=True))

In [294]:
for i in range(10):
    model.fit_generator(batch_feeder(), samples_per_epoch=20000, nb_epoch=1, verbose=0)
    print ("%.2f %.2f" % evaluate_model(model))

0.94 0.83
0.94 0.82
0.93 0.81
0.94 0.83
0.94 0.82
0.94 0.82
0.95 0.83
0.94 0.82
0.94 0.81
0.94 0.82


This model takes the longest to train. The best performance I have seen was 85%.

## Conclusions

Neural networks provide a way to classify nanopore reads before basecalling and without any alignment. Such classification with neural networks are extremely fast, making them ideal for Read Until applications, realtime monitoring and embedded devices.

However, they require a lot of real data. In this project the best out-of-sample performance reached was 85%. This accuracy may be enough to be useful in some applications, but accuracies above 90% would greatly increase usefulness, of course.

Because of other experiments, which I haven't written up yet, I do believe accuracies beyond 90% should be possible in this setting, if more data is available. Specifically, I used other datasets from [Loose et al.] to predict which of five regions of the lambda region a read is from, and achieved an accuracy of up to 95%, from segments of 500 events.


The longer the segment which is used for the classification, the more accurate the classifier can be. In this case I used a segment of size 200 to demonstrate a relatively low baseline. In Read Until applications, the longer the segment used for classification is, the lower the "gain" from the Read Until strategy will be.

In a situation where 10% of the reads are desireable, a Read Until strategy with an accuracy of 80% could increase the desireable output to up to 30%, but only if the reads are long enough.

Neural Networks of this sort are also a good choice for embedded devices with nanopore chips as DNA sensors, because they require significantly less memory and computing power than the basecallers (HMM or RNN) will, but they can still answer simple questions like detect a certain microbe, RNA product or the like.

Other ideas for classification:
* Human vs procaryote
* Genomic vs mitochondrial
* High quality vs low quality/damaged DNA
* Will align/won't align
* Barcoding